<a href="https://colab.research.google.com/github/SakshamDataWizard/Food-wastage-system/blob/main/food_waste_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%writefile food_waste_app.py
import streamlit as st
import sqlite3
import pandas as pd
from datetime import datetime

# Database setup
conn = sqlite3.connect('food_wastage.db')
c = conn.cursor()

# Create tables
c.execute('''CREATE TABLE IF NOT EXISTS food_listings
             (id INTEGER PRIMARY KEY AUTOINCREMENT,
              donor_name TEXT,
              food_type TEXT,
              quantity INTEGER,
              expiration_date DATE,
              location TEXT,
              listing_date DATE,
              claimed BOOLEAN DEFAULT 0)''')

c.execute('''CREATE TABLE IF NOT EXISTS claims
             (claim_id INTEGER PRIMARY KEY AUTOINCREMENT,
              food_id INTEGER,
              claimant_name TEXT,
              claim_date DATE,
              FOREIGN KEY(food_id) REFERENCES food_listings(id))''')
conn.commit()


FOOD_LISTINGS_CSV_PATH = 'food_listings_data.csv'
CLAIMS_CSV_PATH = 'claims_data.csv'

# Load initial data if files exist
try:
    food_df = pd.read_csv("data/food_listings.csv")
    food_df.to_sql('food_listings_data.csv', conn, if_exists='append', index=False)

    claims_df = pd.read_csv("data/claims.csv")
    claims_df.to_sql('claims_data.csv', conn, if_exists='append', index=False)
except FileNotFoundError:
    pass


st.title("Local Food Wastage Management System")

# User type selection
user_type = st.sidebar.selectbox("Select User Type", ["Donor", "Receiver", "Admin"])

if user_type == "Donor":
    st.header("Food Donation Portal")

    with st.form("donation_form"):
        donor_name = st.text_input("Donor Name")
        food_type = st.selectbox("Food Type", ["Vegetarian", "Non-Vegetarian", "Dairy", "Baked Goods"])
        quantity = st.number_input("Quantity (portions)", min_value=1)
        expiration_date = st.date_input("Expiration Date")
        location = st.text_input("Pickup Location")
        submitted = st.form_submit_button("Submit Donation")

        if submitted:
            c.execute('''INSERT INTO food_listings
                        (donor_name, food_type, quantity, expiration_date, location, listing_date)
                        VALUES (?,?,?,?,?,?)''',
                     (donor_name, food_type, quantity, expiration_date, location, datetime.today().date()))
            conn.commit()
            st.success("Donation listed successfully!")

    # Show donor's previous donations
    st.subheader("Your Previous Donations")
    donor_history = c.execute('''SELECT * FROM food_listings
                               WHERE donor_name = ?''', (donor_name,)).fetchall()
    if donor_history:
        donor_df = pd.DataFrame(donor_history,
                              columns=["ID", "Donor", "Type", "Quantity", "Expiration",
                                       "Location", "Listing Date", "Claimed"])
        st.dataframe(donor_df)

        # Delete or update functionality
        selected_id = st.selectbox("Select donation to modify", donor_df['ID'])
        if st.button("Delete Donation"):
            c.execute('DELETE FROM food_listings WHERE id = ?', (selected_id,))
            conn.commit()
            st.success("Donation deleted!")

elif user_type == "Receiver":
    st.header("Available Food Listings")

    # Filter options
    col1, col2 = st.columns(2)
    with col1:
        filter_type = st.selectbox("Filter by Food Type", ["All"] + ["Vegetarian", "Non-Vegetarian", "Dairy", "Baked Goods"])
    with col2:
        filter_location = st.text_input("Filter by Location")

    # Build query
    query = "SELECT * FROM food_listings WHERE claimed = 0"
    params = []

    if filter_type != "All":
        query += " AND food_type = ?"
        params.append(filter_type)

    if filter_location:
        query += " AND location LIKE ?"
        params.append(f"%{filter_location}%")

    available_food = c.execute(query, params).fetchall()

    if available_food:
        food_df = pd.DataFrame(available_food,
                             columns=["ID", "Donor", "Type", "Quantity", "Expiration",
                                      "Location", "Listing Date", "Claimed"])
        st.dataframe(food_df)

        # Claim functionality
        selected_id = st.selectbox("Select food to claim", food_df['ID'])
        claimant_name = st.text_input("Your Name")

        if st.button("Claim Food"):
            c.execute('UPDATE food_listings SET claimed = 1 WHERE id = ?', (selected_id,))
            c.execute('INSERT INTO claims (food_id, claimant_name, claim_date) VALUES (?,?,?)',
                     (selected_id, claimant_name, datetime.today().date()))
            conn.commit()
            st.success("Food claimed successfully!")
    else:
        st.warning("No available food matching your criteria.")

elif user_type == "Admin":
    st.header("Administration Dashboard")

    # Show all data
    st.subheader("All Food Listings")
    full_data = pd.read_sql('SELECT * FROM food_listings', conn)
    st.dataframe(full_data)

    st.subheader("All Claims")
    claims_data = pd.read_sql('SELECT * FROM claims', conn)
    st.dataframe(claims_data)

    # Visualizations
    st.subheader("Food Distribution Analysis")

    col1, col2 = st.columns(2)
    with col1:
        type_dist = pd.read_sql('''SELECT food_type, SUM(quantity) as total_portions
                                  FROM food_listings GROUP BY food_type''', conn)
        st.bar_chart(type_dist.set_index('food_type'))

    with col2:
        location_dist = pd.read_sql('''SELECT location, COUNT(*) as listings
                                      FROM food_listings GROUP BY location''', conn)
        st.write("Listings by Location")
        st.dataframe(location_dist)

    # Map visualization (requires latitude/longitude data)
    # st.map()  # You would need to add geocoding functionality here

conn.close()

Overwriting food_waste_app.py


In [7]:
!pip install streamlit pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.7 MB/s eta 0:00:00


In [3]:
!wget -q -O - ipv4.icanhazip.com

34.125.247.62


In [8]:
! streamlit run food_waste_app.py & npx localtunnel --port 8501


⠙⠹⠸⠼⠴⠦⠧⠇

⠏⠋Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) 
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.247.62:8501

y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸your url is: https://young-words-judge.loca.lt
  Stopping...
^C
